The purpose of this notebook is to incoporate some of Fred Bahr's inital station operation status code into a more modular framework to eventually feed in to a dashboard/UI. <br>
Marine Lebrec, June 2024

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import datetime as dt
from datetime import timedelta
import requests
import json
import sys
import smtplib
from email.message import EmailMessage
import csv

In [6]:
# Get list of all stations that we are interested in.
def get_all_stations():
    station_file = 'station_names.json'
    f = open(station_file)
    data = json.load(f)
    stationnames = []
    for i in data:
        stationnames.append(i['stationName'])
    return stationnames

def get_erddapid(station_file, station_name):
    f = open(station_file)
    data = json.load(f)

    for station in data:
            if station['stationName'] == station_name:
                return station['datasetID']
            
def get_timedelta(erddapID):
    now = dt.datetime.now(tz=dt.timezone.utc)
    df = pd.read_csv(f'https://erddap.sensors.axds.co/erddap/tabledap/{erddapID}.csv?time')
    last_time = dt.datetime.strptime(df['time'].iloc[-1],'%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=dt.timezone.utc)
    time_delta = now - last_time

    # parse for more meaningful output
    days = time_delta.days
    hours, remainder = divmod(time_delta.seconds, 3600)
    minutes, _ = divmod(remainder, 60)

    # Construct the timedelta string without seconds
    if days > 0:
        timedelta_str = f"{days} days, {hours} hours, {minutes} minutes"
    elif hours > 0:
        timedelta_str = f"{hours} hours, {minutes} minutes"
    elif minutes > 0:
        timedelta_str = f"{minutes} minutes"
    else:
        timedelta_str = "< 1 minute"
    
    print(timedelta_str)
    return timedelta_str

def create_clean_csv(outputfile):
     with open(outputfile, 'w', newline='') as csvfile:
            fieldnames = ['stationName', 'timeDelta']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
     
     
def write_to_csv(station, timedelta_str, outputfile):
    with open(outputfile, 'a', newline='') as csvfile:
        fieldnames = ['stationName', 'timeDelta']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({'stationName': station, 'timeDelta': timedelta_str})
        print(f'{station} {timedelta_str} written to csv')

In [8]:
outputfile = 'stations_timedelta.csv'
create_clean_csv(outputfile = outputfile)

for station in get_all_stations():
    erddapid = get_erddapid(station_file = 'station_names.json', station_name = station)
    timedelta_str = get_timedelta(erddapID=erddapid)
    write_to_csv(station = station, timedelta_str = timedelta_str, outputfile = outputfile)

20 hours, 54 minutes
Moss Landing Marine Laboratories Seawater Intake Monitoring Station (MLSC1) 20 hours, 54 minutes written to csv
848 days, 12 hours, 47 minutes
Romberg Tiburon Center Pier 848 days, 12 hours, 47 minutes written to csv
1324 days, 0 hours, 21 minutes
Santa Cruz municipal wharf 1324 days, 0 hours, 21 minutes written to csv
16 minutes
Trinidad Head, California 16 minutes written to csv
230 days, 0 hours, 43 minutes
Bodega Marine Laboratory seawater intake 230 days, 0 hours, 43 minutes written to csv
11 days, 19 hours, 0 minutes
Carquinez 11 days, 19 hours, 0 minutes written to csv
230 days, 4 hours, 13 minutes
Fort Point Pier 230 days, 4 hours, 13 minutes written to csv
16 minutes
Humboldt Bay Pier 16 minutes written to csv
23 hours, 1 minutes
Monterey Wharf Real Time Sampling 23 hours, 1 minutes written to csv
41 minutes
Morro Bay - BM1 T-Pier 41 minutes written to csv
41 minutes
Morro Bay - BS1 41 minutes written to csv
1 hours, 18 minutes
San Luis Bay - Cal Poly Pier

In [72]:
erddapid = get_erddapid(station_file = 'station_names.json', station_name = station)
time_delta = get_timedelta(erddapID=erddapid)
type(time_delta)

datetime.timedelta